In [1]:
# A multi classes image classifier, based on convolutional neural network using Keras and Tensorflow. 
# A multi-label classifier (having one fully-connected layer at the end), with multi-classification (18 classes, in this instance).
# Largely copied from the code https://gist.github.com/seixaslipe
# This is based on these posts: https://medium.com/alex-attia-blog/the-simpsons-character-recognition-using-keras-d8e1796eae36
# Data downloaded from Kaggle 
# Will emulate the image classification functionlities for Neuro Pathology images/slides (WSI-Whole Slide images)
# Will implement/include data manipulating functionalities based on Girder (https://girder.readthedocs.io/en/latest/)
# Has 6 convulsions, filtering start with 64, 128, 256 with flattening to 1024
# Used Keras.ImageDataGenerator for Training/Validation data augmentation and the augmented images are flown from respective directory
# Environment: A docker container having Keras, TensorFlow, Python-2 with GPU based execution

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import Callback
import datetime, time, os, sys
import numpy as np
import h5py
import matplotlib as plt
plt.use('Agg')
import matplotlib.pyplot as pyplot
pyplot.figure
import pickle 
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import subprocess
import pandas as pd

import nvidia_smi as nvs
import io
import pickle
import json

try:
    to_unicode = unicode
except NameError:
    to_unicode = str


Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [ ]:
!pip install nvidia-ml-py --user

In [3]:
modelInfo = {}
modelInfo['Device']  = {} ## Initialize an object to store info on the model and time info

nvs.nvmlInit()

driverVersion = nvs.nvmlSystemGetDriverVersion()
print("Driver Version: {}".format(driverVersion))
modelInfo['Device']['driverVersion']  = driverVersion

# e.g. will print:
#   Driver Version: 352.00
deviceCount = nvs.nvmlDeviceGetCount()
deviceNames = []
for i in range(deviceCount):
    handle = nvs.nvmlDeviceGetHandleByIndex(i)
    dvn = nvs.nvmlDeviceGetName(handle) # store the device name
    print("Device {}: {}".format(i,  dvn))
    deviceNames.append(dvn)
    # e.g. will print:
    #  Device 0 : Tesla K40c
nvs.nvmlShutdown()

modelInfo['Device']['deviceNames']  = deviceNames


### These parameters can be tuned and may affect classification results or accuracy
img_width, img_height = 64, 64
epochs = 50
batch_size = 128


modelInfo['batch_size'] = batch_size
modelInfo['epochs'] = epochs
modelInfo['img_width'] = 64
modelInfo['img_height'] = 64
 

### Define input dirs and output for results which contain the models as well as stats on the run
train_data_dir = '/data/train' 
validation_data_dir = '/data/test' 

resultsDir ="/app/results/"
if not os.path.isdir(resultsDir):
    os.makedirs(resultsDir)

nb_train_samples = 0

for root, dirs, files in os.walk(train_data_dir):
    nb_train_samples += len(files)

nb_validation_samples = 0
for root, dirs, files in os.walk(validation_data_dir):
    nb_validation_samples += len(files)


# Model definition
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip=True)

# Only rescaling for validation
valid_datagen = ImageDataGenerator(rescale=1. / 255.0)

# Flows the data directly from the directory structure, resizing where needed
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

NumLabels = len(validation_generator.class_indices)

'''
6-conv layers - added on 06/21, Raj
'''
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same')) 
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NumLabels, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# Captures GPU usage
#subprocess.Popen("timeout 120 nvidia-smi --query-gpu=utilization.gpu,utilization.memory --format=csv -l 1 | sed s/%//g > /app/results/GPU-stats.log",shell=True)



Driver Version: 384.130
Device 0: GeForce GTX 1050
Found 19548 images belonging to 20 classes.
Found 990 images belonging to 20 classes.


In [ ]:
!pip install nvidia-ml-py --user

In [4]:
# Timehistory callback to get epoch run times
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

# Model fitting and training run
simpsonsModel = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,

    callbacks=[time_callback])    


print "Finished running the basic model... trying to save results now.."


Epoch 1/50
152/152 [==============================] - 106s 695ms/step - loss: 2.7127 - acc: 0.1663 - val_loss: 2.4388 - val_acc: 0.2846
Epoch 2/50
152/152 [==============================] - 98s 642ms/step - loss: 2.0380 - acc: 0.3806 - val_loss: 1.6881 - val_acc: 0.5011
Epoch 3/50
152/152 [==============================] - 97s 638ms/step - loss: 1.5223 - acc: 0.5455 - val_loss: 1.1285 - val_acc: 0.6417
Epoch 4/50
152/152 [==============================] - 97s 641ms/step - loss: 1.1848 - acc: 0.6444 - val_loss: 0.8320 - val_acc: 0.7411
Epoch 5/50
152/152 [==============================] - 97s 635ms/step - loss: 0.9951 - acc: 0.7040 - val_loss: 0.6905 - val_acc: 0.7969
Epoch 6/50
152/152 [==============================] - 95s 628ms/step - loss: 0.8337 - acc: 0.7523 - val_loss: 0.4887 - val_acc: 0.8549
Epoch 7/50
152/152 [==============================] - 99s 650ms/step - loss: 0.7487 - acc: 0.7778 - val_loss: 0.4724 - val_acc: 0.8717
Epoch 8/50
152/152 [==============================] - 

In [5]:
# To write the each epoch run time into a json file
now = datetime.datetime.now()
filetime = str(now.year)+str(now.month)+str(now.day)+'_'+str(now.hour)+str(now.minute)

modelInfo['epochTimeInfo'] = time_callback.times


## Write out the h5/model
modelfilename=resultsDir+'Simpsonsmodel_'+filetime+'.h5'
model.save(modelfilename)

## This outputs the training and validation accuracy and loss functions for each epoch
## This will be graphed as well using plotly ... you can use this data to look for overfitting
## and/or when you can stop training your model because it stops improving
modelInfo['historyData'] =  pd.DataFrame(simpsonsModel.history).to_dict(orient='records')

###target_names maps the character names (or labels) to the index(integer) used in the output files
modelInfo['target_names']  = validation_generator.class_indices

modelInfo['labelname_to_index']  = validation_generator.class_indices
modelInfo['index_to_labelname']  = {(v,k) for k,v in validation_generator.class_indices.iteritems() }

In [6]:
target_names = validation_generator.class_indices

## Prediction for TRAIN data set
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)

##Prediction for TEST data set
y_pred = np.argmax(Y_pred, axis=1)

cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)
cls_rpt = classification_report(validation_generator.classes, y_pred, target_names=target_names) 


In [7]:
print(cls_rpt)  ### This is a 20 by 20 matrix

## This looks cool, but we need to turn it into a table I guess?

                          precision    recall  f1-score   support

charles_montgomery_burns       0.08      0.08      0.08        48
            ned_flanders       0.02      0.02      0.02        50
           homer_simpson       0.08      0.08      0.08        50
           lenny_leonard       0.04      0.04      0.04        48
  abraham_grampa_simpson       0.04      0.04      0.04        50
            mayor_quimby       0.06      0.06      0.06        49
            chief_wiggum       0.06      0.06      0.06        50
          edna_krabappel       0.04      0.04      0.04        50
  apu_nahasapeemapetilon       0.02      0.02      0.02        50
       principal_skinner       0.06      0.06      0.06        50
           marge_simpson       0.04      0.04      0.04        50
             moe_szyslak       0.10      0.10      0.10        50
            nelson_muntz       0.04      0.04      0.04        50
        krusty_the_clown       0.02      0.02      0.02        50
         

In [8]:
## Turning into classification report into classification object
avgresults = cls_rpt.strip().split('\n')[-1].split()

In [9]:
overallResults={'label' : 'avg/total', 'precision': avgresults[3], 'recall':avgresults[4],'f1-score':avgresults[5], 'support':avgresults[6]}


In [10]:
precision, recall, fscore, support  =  precision_recall_fscore_support(validation_generator.classes, y_pred)

In [11]:
modelInfo['classificationObject'] =  characterResultsArray =  {
    'label': validation_generator.class_indices.keys(),
    'precision': precision,
    'recall':recall,
    'fscore': fscore, 'support':support,
    'overallResults':{'label' : 'avg/total', 
                      'precision': avgresults[3], 
                      'recall':avgresults[4],
                      'f1-score':avgresults[5],
                      'support':avgresults[6]}}

In [12]:
modelInfo['classificationObject']

{'fscore': array([0.08247423, 0.01980198, 0.08080808, 0.04123711, 0.04      ,
        0.0625    , 0.06060606, 0.03960396, 0.01980198, 0.06      ,
        0.04040404, 0.1       , 0.04040404, 0.01980198, 0.08      ,
        0.01904762, 0.10416667, 0.02061856, 0.02040816, 0.0212766 ]),
 'label': ['charles_montgomery_burns',
  'ned_flanders',
  'homer_simpson',
  'lenny_leonard',
  'abraham_grampa_simpson',
  'mayor_quimby',
  'chief_wiggum',
  'edna_krabappel',
  'apu_nahasapeemapetilon',
  'principal_skinner',
  'marge_simpson',
  'moe_szyslak',
  'nelson_muntz',
  'krusty_the_clown',
  'kent_brockman',
  'bart_simpson',
  'sideshow_bob',
  'comic_book_guy',
  'lisa_simpson',
  'milhouse_van_houten'],
 'overallResults': {'f1-score': u'0.05',
  'label': 'avg/total',
  'precision': u'0.05',
  'recall': u'0.05',
  'support': u'990'},
 'precision': array([0.08163265, 0.01960784, 0.08163265, 0.04081633, 0.04      ,
        0.06382979, 0.06122449, 0.03921569, 0.01960784, 0.06      ,
        0.

In [13]:
modelInfo['confusion_matrix'] = confusion_matrix(validation_generator.classes, y_pred)

In [14]:
modelInfo['confusion_matrix']

array([[4, 2, 3, 1, 4, 2, 2, 1, 5, 4, 1, 1, 4, 2, 2, 2, 2, 1, 1, 4],
       [2, 1, 2, 1, 3, 3, 1, 6, 2, 3, 2, 3, 2, 4, 1, 2, 2, 4, 4, 2],
       [0, 4, 4, 4, 4, 4, 0, 2, 2, 4, 2, 0, 3, 1, 5, 1, 5, 3, 2, 0],
       [5, 6, 2, 2, 4, 1, 3, 3, 1, 0, 2, 2, 2, 4, 1, 1, 2, 4, 1, 2],
       [5, 2, 0, 4, 2, 2, 0, 4, 2, 3, 2, 2, 0, 2, 3, 6, 3, 1, 4, 3],
       [2, 1, 1, 2, 2, 3, 3, 1, 1, 3, 5, 2, 3, 3, 5, 3, 1, 1, 4, 3],
       [3, 4, 1, 3, 4, 3, 3, 2, 4, 1, 3, 2, 5, 3, 0, 2, 0, 2, 2, 3],
       [3, 3, 2, 1, 0, 3, 3, 2, 4, 1, 3, 1, 2, 2, 2, 7, 3, 3, 2, 3],
       [1, 2, 3, 1, 2, 2, 2, 4, 1, 1, 0, 1, 3, 4, 4, 2, 3, 5, 4, 5],
       [1, 4, 4, 2, 2, 1, 4, 4, 5, 3, 2, 3, 2, 1, 2, 5, 2, 2, 0, 1],
       [3, 2, 1, 3, 2, 1, 3, 1, 4, 2, 2, 1, 4, 4, 1, 4, 3, 2, 3, 4],
       [1, 0, 2, 5, 1, 3, 1, 7, 2, 4, 3, 5, 3, 3, 1, 2, 2, 1, 2, 2],
       [2, 3, 5, 4, 4, 2, 3, 0, 2, 2, 5, 2, 2, 4, 2, 5, 2, 0, 1, 0],
       [3, 2, 2, 3, 3, 2, 0, 2, 4, 2, 3, 9, 2, 1, 1, 3, 0, 3, 2, 3],
       [1, 2, 3, 3, 2, 3, 4, 3, 2,

In [15]:
###   filename --- CLASS


### LAST BUT NOT LEAST --- 



# MAKE IT A PARAMETER OUTPUT MODELPREDICTIOJ FOR TRAIN AND TEST OR JUST TEST  

for image in glob(train/*):
    I WANT
    
    ['filename': "somename", 'actualImageLabel': asIndex, 'modelPrection': X ]
    
   


SyntaxError: invalid syntax (<ipython-input-15-44a298889d69>, line 10)